<a href="https://colab.research.google.com/github/Sukanya41455/Tensorflow_practice/blob/main/pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

In [10]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X,y in test_dataloader:
  print(f"Shape of X [N, C, H, W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

cpu
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
# optimizing model params
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # backpropogation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [14]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [15]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-----------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------------------
loss: 2.317462 [    0/60000]
loss: 2.300936 [ 6400/60000]
loss: 2.278818 [12800/60000]
loss: 2.255799 [19200/60000]
loss: 2.245684 [25600/60000]
loss: 2.222616 [32000/60000]
loss: 2.233062 [38400/60000]
loss: 2.210456 [44800/60000]
loss: 2.195853 [51200/60000]
loss: 2.149729 [57600/60000]
Test error: 
 Accuracy: 38.3%, Avg loss: 2.154625

Epoch 2
-----------------------
loss: 2.182267 [    0/60000]
loss: 2.168261 [ 6400/60000]
loss: 2.104367 [12800/60000]
loss: 2.100064 [19200/60000]
loss: 2.053178 [25600/60000]
loss: 2.002659 [32000/60000]
loss: 2.033689 [38400/60000]
loss: 1.965223 [44800/60000]
loss: 1.962531 [51200/60000]
loss: 1.872918 [57600/60000]
Test error: 
 Accuracy: 49.6%, Avg loss: 1.881242

Epoch 3
-----------------------
loss: 1.935456 [    0/60000]
loss: 1.902611 [ 6400/60000]
loss: 1.780198 [12800/60000]
loss: 1.797424 [19200/60000]
loss: 1.680118 [25600/60000]
loss: 1.652730 [32000/60000]
loss: 1.673922 [38400/60000]
loss: 1.587349 [448

In [16]:
# saving model
torch.save(model.state_dict(), "model.pth")
print("Saved model to model.pth")

Saved model to model.pth


In [17]:
# loading models
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
